## Część 2

In [74]:
import os
import pandas as pd
from google.cloud import bigquery
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="secret.json" # lokalizacja pobranego klucza z punktu 1.4.
client = bigquery.Client()
query = ('select * from bigquery-public-data.covid19_open_data.covid19_open_data limit 10')
query_job = client.query(query)
query_result = query_job.result()
df1 = query_result.to_dataframe()
df1

,location_key,date,place_id,wikidata_id,datacommons_id,country_code,country_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,aggregation_level,...,cumulative_vaccine_doses_administered_pfizer,new_persons_fully_vaccinated_moderna,cumulative_persons_fully_vaccinated_moderna,new_vaccine_doses_administered_moderna,cumulative_vaccine_doses_administered_moderna,new_persons_fully_vaccinated_janssen,cumulative_persons_fully_vaccinated_janssen,new_vaccine_doses_administered_janssen,cumulative_vaccine_doses_administered_janssen,location_geometry
0,AR,2020-12-22,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
1,AR,2021-11-12,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
2,AR,2021-02-26,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
3,AR,2021-02-22,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
4,AR,2020-03-11,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
5,AR,2022-05-04,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
6,AR,2021-08-13,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
7,BR,2020-07-31,ChIJzyjM68dZnAARYz4p8gYVWik,Q155,country/BRA,BR,Brazil,BR,BRA,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-53 -14)
8,BR,2020-01-12,ChIJzyjM68dZnAARYz4p8gYVWik,Q155,country/BRA,BR,Brazil,BR,BRA,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-53 -14)
9,BR,2020-06-17,ChIJzyjM68dZnAARYz4p8gYVWik,Q155,country/BRA,BR,Brazil,BR,BRA,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-53 -14)


## Część 3

3.1. Sprawdź, ile jest zapisanych wierszy z danymi. 

In [75]:
query = ("""
select count(*) as liczba_wierszy from bigquery-public-data.covid19_open_data.covid19_open_data;
""")

query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,liczba_wierszy
0,22756333


 3.2. Sprawdź, ile krajów jest uwzględnionych w danych. 



In [76]:
query = ("""
select count(*) as liczba_krajow FROM (SELECT DISTINCT country_name FROM bigquery-public-data.covid19_open_data.covid19_open_data);
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,liczba_krajow
0,246


 3.3. Sprawdź, w jaki sposób zapisywane są dzienne informacje dla krajów.



Dzienne informacje dla krajów raportowane są z podziałem na części opisywane przez aggregation_level.
Dane odnośnie osób hospitalizowanych, zaszczepionych, zmarłych i testowanych raportowane są oddzielnie dla poszczególnych przedziałów wiekowych oraz z podziałem na płeć. 
Dane zawierają również podstawowe informacje o poszczególnych krajach, takie jak populacja, powierzchnia, wilgotność czy przewidywana długość życia.

 3.4. Sprawdź, w jaki sposób zapisywane są wartości liczbowe.  



In [77]:
print(df1['population'].dtype)
print(df1['nurses_per_1000'].dtype)
print(df1['new_tested'].dtype)

Int64
float64
object


 3.5. Sprawdź, jaki przedział czasowy jest uwzględniony w danych. Dodatkowo porównaj przedziały czasowe dla przypadków nowych zachorowań, nowych śmierci oraz nowych zaszczepionych osób w danych.



In [78]:
query = ("""
select min(date) as poczatek_przedzialu, max(date) as koniec_przedzialu FROM bigquery-public-data.covid19_open_data.covid19_open_data;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df
# co to znaczy nowych?

,poczatek_przedzialu,koniec_przedzialu
0,2020-01-01,2022-09-17


In [79]:
query = ("""
select min(date) as poczatek_przedzialu_zachorowan, max(date) as koniec_przedzialu_zachorowan FROM bigquery-public-data.covid19_open_data.covid19_open_data where new_confirmed is not null and new_confirmed <> 0;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,poczatek_przedzialu_zachorowan,koniec_przedzialu_zachorowan
0,2020-01-01,2022-09-15


In [80]:
query = ("""
select min(date) as poczatek_przedzialu_smierci, max(date) as koniec_przedzialu_smierci FROM bigquery-public-data.covid19_open_data.covid19_open_data where new_deceased is not null and new_deceased <> 0;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,poczatek_przedzialu_smierci,koniec_przedzialu_smierci
0,2020-01-02,2022-09-15


In [81]:
query = ("""
select min(date) as poczatek_przedzialu_zaszczepionych, max(date) as koniec_przedzialu_zaszczepionych FROM bigquery-public-data.covid19_open_data.covid19_open_data where new_persons_vaccinated is not null and new_persons_vaccinated <> 0;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,poczatek_przedzialu_zaszczepionych,koniec_przedzialu_zaszczepionych
0,2020-12-07,2022-09-14


 3.6. Sprawdź więcej informacji (co najmniej 5 różnych) o danych dotyczących COVID-19. W tym celu nie wykonuj żadnych dodatkowych obliczeń. 



In [82]:
# Sprawdzenie czy wystepuja wartosci ujemne
query = ("""
select COUNT(*) FROM bigquery-public-data.covid19_open_data.covid19_open_data where new_confirmed < 0;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,f0_
0,102199


In [83]:
#Sprawdzenie czy wystepuja duplikaty
query = ("""
select COUNT(*), location_key, date FROM bigquery-public-data.covid19_open_data.covid19_open_data group by location_key, date having count(*) > 1;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,f0_,location_key,date


In [84]:
# Sprawdzenie czy kluczowe kolumny zawieraja nulle
query = ("""
select COUNT(*) FROM bigquery-public-data.covid19_open_data.covid19_open_data where date is null or country_name is null or location_key is null or iso_3166_1_alpha_3 is null;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,f0_
0,991


In [85]:
# Sprawdzenie z jakich krajow posiadamy najwiecej danych
query = ("""
select country_name, count(*) as count from bigquery-public-data.covid19_open_data.covid19_open_data group by country_name order by count desc limit 10;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,country_name,count
0,Brazil,5565456
1,United States of America,3255435
2,Mexico,2475518
3,Peru,1883891
4,Israel,1472626
5,Spain,1385418
6,Colombia,1145596
7,India,747214
8,Indonesia,544059
9,Argentina,540095


In [86]:
# sprawdzenie czy w kolumnach cumulative wystepuja wartosci mniejsze niz w kolumnach new
query = ("""
select distinct location_key, country_name from bigquery-public-data.covid19_open_data.covid19_open_data where (new_confirmed > cumulative_confirmed or new_deceased > cumulative_deceased or cast(new_tested as int) > cumulative_tested or new_persons_vaccinated > cumulative_persons_vaccinated or new_persons_fully_vaccinated > cumulative_persons_fully_vaccinated or new_vaccine_doses_administered > cumulative_vaccine_doses_administered);
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,location_key,country_name
0,PL_04,Poland
1,PL_18,Poland
2,CH_GE,Switzerland
3,US_OK,United States of America
4,PL_10,Poland
5,PL_28,Poland
6,PL_02,Poland
7,PL_14,Poland
8,PL_22,Poland
9,PL_16,Poland


## Część 4

4.1. Chcemy posiadać podstawowe dane o wszystkich krajach świata, które będą zrozumiałe dla człowieka oraz uniwersalne i potencjalnie przyszłościowe do dalszego przetwarzania.



In [87]:
query = ("""
SELECT distinct
    location_key,
    subregion1_code,
    subregion1_name,
    country_code,
    country_name, 
    iso_3166_1_alpha_2,
    iso_3166_1_alpha_3, 
    population, 
    population_male, 
    population_female, 
    population_density, 
    human_development_index, 
    gdp_usd, 
    gdp_per_capita_usd, 
    latitude, 
    longitude, 
    area_sq_km, 
    health_expenditure_usd, 
    out_of_pocket_health_expenditure_usd 
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE 
    location_key IS NOT NULL AND
    country_code IS NOT NULL AND 
    country_name IS NOT NULL AND 
    iso_3166_1_alpha_2 IS NOT NULL AND 
    iso_3166_1_alpha_3 IS NOT NULL AND 
    population IS NOT NULL AND 
    population_male IS NOT NULL AND 
    population_female IS NOT NULL AND 
    population_density IS NOT NULL AND 
    human_development_index IS NOT NULL AND 
    gdp_usd IS NOT NULL AND 
    gdp_per_capita_usd IS NOT NULL AND 
    latitude IS NOT NULL AND 
    longitude IS NOT NULL AND 
    area_sq_km IS NOT NULL AND 
    health_expenditure_usd IS NOT NULL AND 
    out_of_pocket_health_expenditure_usd IS NOT NULL
    AND aggregation_level = 0;
""")
query_job = client.query(query)
query_result = query_job.result()
df1 = query_result.to_dataframe()
df1.to_csv("countries_data.csv", index=False)
df1

,location_key,subregion1_code,subregion1_name,country_code,country_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,population,population_male,population_female,population_density,human_development_index,gdp_usd,gdp_per_capita_usd,latitude,longitude,area_sq_km,health_expenditure_usd,out_of_pocket_health_expenditure_usd
0,BD,None,None,BD,Bangladesh,BD,BGD,164689383,83259108,81430275,1265.187,0.632,302571254131,1855,24.016667,89.866667,147630,36.282337,26.807108
1,CM,None,None,CM,Cameroon,CM,CMR,26545864,13277075,13268789,56.157,0.556,38760467033,1497,7.000000,12.000000,475440,67.811813,48.117634
2,GA,None,None,GA,Gabon,GA,GAB,2225728,1132854,1092874,8.638,0.702,16657960228,7667,-0.683331,11.500000,267670,204.492249,51.442871
3,HR,None,None,HR,Croatia,HR,HRV,4076246,1972572,2103674,73.200,0.831,60917594400,14868,45.250000,15.466667,56590,902.139648,98.994057
4,KN,None,None,KN,Saint Kitts and Nevis,KN,KNA,53192,30289,30759,204.585,0.778,1050992592,19896,17.271667,-62.666669,260,902.658997,431.199463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,PL,None,None,PL,Poland,PL,POL,37972812,18380376,19592436,123.600,0.843,585705873800,15222,52.000000,19.000000,312680,906.820129,209.020157
171,QA,None,None,QA,Qatar,QA,QAT,2881060,2165135,715925,248.153,0.856,183466208791,64781,25.269535,51.212767,11610,1649.186157,146.821411
172,RU,None,None,RU,Russia,RU,RUS,145934460,67640299,78294161,8.911,0.816,1699876578871,11584,66.416667,94.250000,17098250,585.873230,237.238129
173,RW,None,None,RW,Rwanda,RW,RWA,12952209,6367431,6584778,525.019,0.524,10122472590,801,-1.940278,29.873889,26340,49.204456,3.074294


 4.2. Chcemy wygenerować statystyki dotyczące zachorowań na COVID-19 na całym świecie.



In [88]:
query = ("""select distinct location_key, subregion1_code, subregion1_name,date, country_name, new_confirmed, cumulative_confirmed, cast(new_tested as int) as new_tested,  cumulative_tested from bigquery-public-data.covid19_open_data.covid19_open_data WHERE
    location_key IS NOT NULL AND
  date IS NOT NULL
  AND country_name IS NOT NULL
  AND new_confirmed IS NOT NULL
  AND cumulative_confirmed IS NOT NULL
  AND new_tested IS NOT NULL
  AND cumulative_tested IS NOT NULL
  AND new_confirmed <= cumulative_confirmed
  and cast(new_tested as int) <= cumulative_tested
  AND new_confirmed >= 0
  AND cast(new_tested as int) >= 0
  AND cumulative_confirmed >= 0
  AND cumulative_tested >= 0
  """)
query_job = client.query(query)
query_result = query_job.result()
df2 = query_result.to_dataframe()
df2.to_csv("covid_cases_stats.csv", index=False)
df2

,location_key,subregion1_code,subregion1_name,date,country_name,new_confirmed,cumulative_confirmed,new_tested,cumulative_tested
0,AR_C,C,City of Buenos Aires,2020-05-30,Argentina,348,8141,1084,32943
1,AR_E,E,Entre Ríos,2021-12-29,Argentina,426,139483,1002,268563
2,AR_J,J,San Juan,2020-09-22,Argentina,19,602,26,2071
3,AR_P,P,Formosa,2021-09-19,Argentina,17,61789,1341,513854
4,AR_Q,Q,Neuquén,2020-03-23,Argentina,0,10,4,38
...,...,...,...,...,...,...,...,...,...
2961099,IN_WB_MPN,WB,West Bengal,2020-07-20,India,0,64,0,0
2961100,IN_WB_PRL,WB,West Bengal,2022-05-11,India,0,24234,0,0
2961101,IN_WB,WB,West Bengal,2021-11-10,India,853,1600732,42113,19572879
2961102,IN_WB_DPD,WB,West Bengal,2021-05-01,India,107,9912,0,0


 4.3. Chcemy poznać efekty COVID-19 poprzez uwypuklenie problemu śmiertelności ludzi spowodowanej wirusem.



In [89]:
query = ("""
SELECT 
    location_key,
    subregion1_code,
    subregion1_name,
    date,  
    country_name,  
    new_deceased, 
    cumulative_deceased 
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE 
    date IS NOT NULL AND  
    location_key IS NOT NULL AND
    country_name IS NOT NULL AND 
    new_deceased IS NOT NULL AND 
    cumulative_deceased IS NOT NULL AND
    new_deceased <= cumulative_deceased AND
    new_deceased >= 0
    AND cumulative_deceased >= 0
    ;

""")
query_job = client.query(query)
query_result = query_job.result()
df3 = query_result.to_dataframe()
df3.to_csv("covid_mortality.csv", index=False)
df3

,location_key,subregion1_code,subregion1_name,date,country_name,new_deceased,cumulative_deceased
0,US_MT_30077,MT,Montana,2021-03-04,United States of America,0,7
1,US_NJ_34001,NJ,New Jersey,2022-04-01,United States of America,0,989
2,US_NY_36033,NY,New York,2022-03-30,United States of America,0,44
3,US_NY_36073,NY,New York,2021-10-15,United States of America,1,87
4,US_NY_36085,NY,New York,2020-10-07,United States of America,0,885
...,...,...,...,...,...,...,...
11440092,US_VA_51157,VA,Virginia,2020-12-28,United States of America,0,2
11440093,US_VA_51137,VA,Virginia,2020-11-20,United States of America,0,7
11440094,US_VA_51600,VA,Virginia,2022-04-09,United States of America,0,42
11440095,US_VA_51041,VA,Virginia,2021-10-07,United States of America,1,511


 4.4. Chcemy zaobserwować trendy i zależności dotyczące szczepień na COVID-19.



In [90]:

import numpy as np

query = ("""
SELECT 
    location_key,
    subregion1_code,
    subregion1_name,
    date,  
    country_name, 
    new_persons_vaccinated, 
    cumulative_persons_vaccinated, 
    new_persons_fully_vaccinated, 
    cumulative_persons_fully_vaccinated, 
    new_vaccine_doses_administered, 
    cumulative_vaccine_doses_administered 
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE 
    location_key IS NOT NULL AND
    date IS NOT NULL AND  
    country_name IS NOT NULL AND
    new_persons_vaccinated >= 0 AND
    cumulative_persons_vaccinated >= 0 AND
    new_persons_fully_vaccinated >= 0 AND
    cumulative_persons_fully_vaccinated >= 0 AND
    new_vaccine_doses_administered >= 0 AND
    cumulative_vaccine_doses_administered >= 0
    ;

""")
query_job = client.query(query)
query_result = query_job.result()
df4 = query_result.to_dataframe()
df4.replace(np.nan, 0, inplace=True)
df4.to_csv("covid_vaccination.csv", index=False)
df4

,location_key,subregion1_code,subregion1_name,date,country_name,new_persons_vaccinated,cumulative_persons_vaccinated,new_persons_fully_vaccinated,cumulative_persons_fully_vaccinated,new_vaccine_doses_administered,cumulative_vaccine_doses_administered
0,AR_D,D,San Luis,2021-12-04,Argentina,12,191875,244,173722,256,365597
1,AR_D,D,San Luis,2021-04-19,Argentina,17,32723,7,6005,24,38728
2,AR_D,D,San Luis,2021-12-05,Argentina,4,191879,11,173733,15,365612
3,AR_D,D,San Luis,2022-03-09,Argentina,51,201775,84,190342,135,392117
4,AR_D,D,San Luis,2021-11-10,Argentina,279,187924,1415,153406,1694,341330
...,...,...,...,...,...,...,...,...,...,...,...
877016,PL_10_62,10,Łódź,2021-05-06,Poland,657,18711,339,6867,996,25578
877017,PL_10_62,10,Łódź,2021-12-30,Poland,67,49267,75,41766,585,107394
877018,PL_10_62,10,Łódź,2022-01-09,Poland,26,49629,25,42143,239,110242
877019,PL_10_62,10,Łódź,2021-06-16,Poland,388,32117,675,17699,1063,49816


 4.5. Zdefiniuj własny dodatkowy przypadek. Chcemy dowiedzieć się ile jest pielęgniarek i lekarzy w poszczególnych krajach.



In [91]:
query = ("""
SELECT distinct
    location_key,
    subregion1_code,
    subregion1_name,
    country_name, 
    nurses_per_1000,
    physicians_per_1000
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE 
    location_key IS NOT NULL AND
    country_name IS NOT NULL AND 
    nurses_per_1000 >= 0 AND
    physicians_per_1000 >= 0
    ;
""")
query_job = client.query(query)
query_result = query_job.result()
df5 = query_result.to_dataframe()
df5.to_csv("covid_vaccination.csv", index=False)
df5

,location_key,subregion1_code,subregion1_name,country_name,nurses_per_1000,physicians_per_1000
0,AO,None,None,Angola,0.4075,0.2146
1,CO,None,None,Colombia,1.3309,2.1848
2,IR,None,None,Iran,2.6286,1.5844
3,IT,None,None,Italy,5.7401,3.9774
4,KW,None,None,Kuwait,7.4145,2.6463
...,...,...,...,...,...,...
159,SC,None,None,Seychelles,8.0773,2.1212
160,SN,None,None,Senegal,0.3127,0.0691
161,SR,None,None,Suriname,2.7569,1.2101
162,WS,None,None,Samoa,2.4885,0.3445


## Część 5
Połącz ze sobą wszystkie dane otrzymane w części 4. Nowy zbiór danych zapisz jako oddzielny obiekt DataFrame. Jeżeli uważasz, że należy stworzyć kilka takich obiektów, zrób to i zapisz swoje uzasadnienie. Pamiętaj o ciągłej konieczności zachowania czystości danych. Wynik tego zadania zapisz w pliku / plikach CSV.

In [92]:
# df1 + gdp + world_countries + df5
# df2 + df3 + df4
merged_df = df2.merge(df3, on=["location_key", "date"], how="inner")
merged_df = merged_df.merge(df4, on=["location_key", "date"], how="inner")
merged_df = merged_df.dropna()
merged_df.to_csv("covid_combined_data.csv", index=False)
merged_df

,location_key,subregion1_code_x,subregion1_name_x,date,country_name_x,new_confirmed,cumulative_confirmed,new_tested,cumulative_tested,subregion1_code_y,...,cumulative_deceased,subregion1_code,subregion1_name,country_name,new_persons_vaccinated,cumulative_persons_vaccinated,new_persons_fully_vaccinated,cumulative_persons_fully_vaccinated,new_vaccine_doses_administered,cumulative_vaccine_doses_administered
0,AR_E,E,Entre Ríos,2021-12-29,Argentina,426,139483,1002,268563,E,...,2493,E,Entre Ríos,Argentina,708,482053,1141,407235,1849,889288
1,AR_P,P,Formosa,2021-09-19,Argentina,17,61789,1341,513854,P,...,1175,P,Formosa,Argentina,69,186132,1797,117656,1866,303788
2,IN_AP,AP,Andhra Pradesh,2021-10-17,India,432,2060472,31712,28985846,AP,...,14307,AP,Andhra Pradesh,India,15773,30038693,22777,16658462,38550,46697155
3,MY_05,05,Negeri Sembilan,2022-01-30,Malaysia,204,119184,2970,1195785,05,...,1334,05,Negeri Sembilan,Malaysia,3,959164,5,951439,2109,2390838
4,AR_A_028,A,Salta,2021-01-02,Argentina,13,10738,86,23450,A,...,571,A,Salta,Argentina,121,705,0,0,121,705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350782,IN_WB,WB,West Bengal,2021-01-28,India,289,569173,25129,7919637,WB,...,10148,WB,West Bengal,India,29292,187485,0,0,29292,187485
350783,IN_WB,WB,West Bengal,2021-07-28,India,815,1525773,45122,15599120,WB,...,18109,WB,West Bengal,India,178950,20136326,84958,8461201,263908,28597527
350784,IN_WB,WB,West Bengal,2021-01-31,India,229,569998,25046,7995854,WB,...,10173,WB,West Bengal,India,74,243143,0,0,74,243143
350785,IN_WB,WB,West Bengal,2021-03-28,India,827,584666,26545,9113077,WB,...,10324,WB,West Bengal,India,1173,4230496,116,636238,1289,4866734


## Część 6
Połącz ze sobą dane otrzymane w części 5 oraz dane znajdujące się w plikach

In [93]:
world_countries = pd.read_csv("world_countries.csv")
world_countries = world_countries.rename(columns={"CCA3": "iso_3166_1_alpha_3", "Country/Territories": "country_name", "Area (km²)": "area_sq_km", "Density (per km²)": "population_density"})

gdp = pd.read_csv("GDP.csv")
gdp = gdp.rename(columns={"Country Code": "iso_3166_1_alpha_3", "Country Name": "country_name", "Value": "gdp_usd", "Year":"year"})
df1["year"] = pd.to_datetime(merged_df["date"]).dt.year
merged_df2 = df1.merge(world_countries, on="iso_3166_1_alpha_3", how="inner")
merged_df2 = merged_df2.merge(df5, on=['country_name'])
pivot_gdp = gdp.pivot(index='country_name', columns='year', values='gdp_usd')
pivot_gdp.columns = ['gdp_' + str(column) for column in pivot_gdp.columns]
merged_df2 = merged_df2.merge(pivot_gdp, on="country_name", how="inner")
merged_df2.to_csv("countries_combined_data.csv", index=False)
merged_df2


,location_key_x,subregion1_code_x,subregion1_name_x,country_code,country_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,population,population_male,population_female,...,gdp_2007,gdp_2008,gdp_2009,gdp_2010,gdp_2011,gdp_2012,gdp_2013,gdp_2014,gdp_2015,gdp_2016
0,BD,None,None,BD,Bangladesh,BD,BGD,164689383,83259108,81430275,...,7.961189e+10,9.163128e+10,1.024778e+11,1.152791e+11,1.286379e+11,1.333557e+11,1.499905e+11,1.728855e+11,1.950787e+11,2.214152e+11
1,GA,None,None,GA,Gabon,GA,GAB,2225728,1132854,1092874,...,1.243896e+10,1.550857e+10,1.206514e+10,1.435859e+10,1.818648e+10,1.717145e+10,1.759075e+10,1.817972e+10,1.426203e+10,1.421356e+10
2,HR,None,None,HR,Croatia,HR,HRV,4076246,1972572,2103674,...,6.009316e+10,7.048145e+10,6.270310e+10,5.966543e+10,6.223675e+10,5.648530e+10,5.776987e+10,5.708037e+10,4.892188e+10,5.071496e+10
3,LK,None,None,LK,Sri Lanka,LK,LKA,21413250,10267351,11145899,...,3.235025e+10,4.071381e+10,4.206622e+10,5.672575e+10,6.529274e+10,6.843440e+10,7.431781e+10,7.935646e+10,8.061199e+10,8.132188e+10
4,NA,None,None,NA,Namibia,NA,NAM,2540916,1231683,1309233,...,8.740866e+09,8.486722e+09,8.876191e+09,1.128219e+10,1.240963e+10,1.301627e+10,1.271779e+10,1.278608e+10,1.157114e+10,1.094788e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,KM,None,None,KM,Comoros,KM,COM,869595,438653,430942,...,4.624536e+08,5.231349e+08,5.241573e+08,5.304934e+08,5.862818e+08,5.708659e+08,6.186639e+08,6.477207e+08,5.656898e+08,6.166545e+08
133,PL,None,None,PL,Poland,PL,POL,37972812,18380376,19592436,...,4.292496e+11,5.338158e+11,4.403466e+11,4.792579e+11,5.287251e+11,5.002840e+11,5.242012e+11,5.450759e+11,4.772796e+11,4.713644e+11
134,QA,None,None,QA,Qatar,QA,QAT,2881060,2165135,715925,...,7.971209e+10,1.152701e+11,9.779835e+10,1.251223e+11,1.677753e+11,1.868335e+11,1.987277e+11,2.062247e+11,1.646415e+11,1.524519e+11
135,RW,None,None,RW,Rwanda,RW,RWA,12952209,6367431,6584778,...,3.824812e+09,4.860577e+09,5.379378e+09,5.774004e+09,6.491684e+09,7.315702e+09,7.622526e+09,8.016288e+09,8.261034e+09,8.376049e+09
